In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from scipy.optimize import bracket
np.random.seed(214)

In [ ]:
cond_number = 30
M = np.random.randn(2, 2)
M = np.dot(M, M.T)
U, s, V = np.linalg.svd(M)
s = np.linspace(cond_number, 1, len(s))
A = np.dot(U, np.dot(np.diag(s), V))
b = np.random.randn(2)

In [ ]:
def rosenbrock_function(x):
    return 10 * (x[1] - x[0]**2)**2 + (x[0] - 1)**2

def grad_rosenbrock(x):
    return np.array([40 * x[0]**3 + (2 - 40 * x[1]) * x[0] - 2, 20 * (x[1] - x[0]**2)]) 

def quadratic_function(x):
    return 0.5 * np.dot(x.T, np.dot(A, x)) - np.dot(b.T, x)

def grad_quadratic(x):
    return np.dot(A, x) - b

In [ ]:
func_name = 'rosenbrock'

In [ ]:
if func_name == 'rosenbrock'
    f = rosenbrock_function
    gradf = grad_rosenbrock
    alpha_0 = 0.06
    beta_0 = 0.1
    c_1 = 1e-4 
    c_2 = 0.99
elif func_name == 'quadratic':
    f = quadratic_function
    gradf = grad_quadratic
    alpha_0 = 0.3
    beta_0 = 0.8
    c_1 = 1e-4
    c_2 = 0.9

In [ ]:
rho = 0.75
epsilon = 1e-2

In [ ]:
def gradient_descent(start_point, stepsize_func, max_iter=1000):
    x = start_point.copy()
    trajectory = [x.copy()]
    for _ in range(max_iter):
        grad = gradf(x)
        step_size = stepsize_func(x, grad)
        x -= step_size * grad
        trajectory.append(x.copy())
    return np.array(trajectory)

In [ ]:

def backtracking_line_search(x, grad, method='default', alpha=alpha_0, beta=beta_0):
    def objective(t):
        return rosenbrock_function(x - t * grad)

    def sufficient_decrease_condition():
        return objective(alpha) <= rosenbrock_function(x) - c_1 * alpha * grad.T @ grad 

    def curvature_condition():
        return grad_rosenbrock(x - alpha * grad).T @ grad <= c_2 * grad.T @ grad 
   
    def wolfe_condition():
        return sufficient_decrease_condition() and curvature_condition()

    def goldstein_condition():
        return objective(alpha) >= rosenbrock_function(x) - rho * alpha * grad.T @ grad \
           and objective(alpha) <= rosenbrock_function(x) - (1 - rho) * alpha * grad.T @ grad

    def dichotomy():
        a, _, b, _, _, _, _ = bracket(objective)
        c = (a + b) / 2
        while abs(b - a) > epsilon:
            y = (a + c) / 2
            if objective(y) <= objective(c):
                b, c = c, y 
            else:
                z = (b + c) / 2
                if objective(c) <= objective(z):
                    a, b = y, z
                else:
                    a, c = c, z
        return c

    if method == 'default':
        res = minimize_scalar(objective, method='golden')
        return res.x
    elif method == 'dichotomy':
        return dichotomy() 
    elif method == 'wolfe':
        condition = wolfe_condition
    elif method == 'curvature':
        condition = curvature_condition
    elif method == 'goldstein':
        condition = goldstein_condition
    elif method == 'sufficient_decrease':
        condition = sufficient_decrease_condition
    else:
        raise Exception('unsupported method')

    while not condition():
        alpha *= beta
    return alpha